In [1]:
from numpy import * 
from __future__ import division
import xlrd
import collections
class Fencing:
    def __init__(self,a,b,c,d,e,f):
        self.ritokuhyou = xlrd.open_workbook('ritokuhyou_update.xlsx').sheet_by_index(0)
        self.point= []#point　= [[一回目のplayer1の得点,一回目のplayer2の得点],[2回目のplayer1の得点,2回目のplayer2の得点]と続いていく]
        self.t = 1 #今何期か
        self.kaisu_player_one_victory = 0  #player1の勝利回数
        self.player_one_point = 0 #player1の点数
        self.player_two_point = 0 #player2の点数
        self.disgise_one = array([[0.5,0.5/3,0.5/3,0.5/3],[0.5/3,0.5,0.5/3,0.5/3],[0.5/3,0.5/3,0.5,0.5/3],[0.5/3,0.5/3,0.5/3,0.5]])
        #上の一行はplayer1の４種類の初動動作に対してにそれぞれ、順に「Aを打ちやすいマルシェ」、「Pを打ちやすいマルシェ」、「Bを打ちやすいマルシェ」、「Rを打ちやすいマルシェ」とplayer2が見る確率
        self.disgise_two = array([[0.5,0.5/3,0.5/3,0.5/3],[0.5/3,0.5,0.5/3,0.5/3],[0.5/3,0.5/3,0.5,0.5/3],[0.5/3,0.5/3,0.5/3,0.5]])
        self.choice_probability_one =array([a,b,c,1-a-b-c])#player1が0,1,2,3をだす確率。０が「Aを打ちやすいマルシェ」、１が「Pを打ちやすいマルシェ」、２が「Bを打ちやすいマルシェ」、３が「Rを打ちやすいマルシェ」に対応している。
        self.choice_probability_two =array([d,e,f,1-d-e-f])
        self.reflection_probability_one = [[0.25,0.25,0.25,0.25],[0.25,0.25,0.25,0.25],[1,0,0,0],[1,0,0,0]]#player2が最終動作に出す手の予想[A,P,B,R]それぞれに対応してplayer1が0,1,2,3を返す確率。0,1,2,3は[A,P,B,R]に対応している。
        self.reflection_probability_two = [[0,0,0,1],[0,0,0,1],[1,0,0,0],[1,0,0,0]]
        self.koho=["a","b","r","p"]
        self.henkan= {"a":0,"p":1,"b":2,"r":3}
        self.gyaku_henkan = {0:"a",1:"p",2:"b",3:"r"}
        self.player_two_record= [[],[],[],[]]  #APBRの順  
        self.player_one_record= [[],[],[],[]]  #APBRの順  
        self.switch_one = False
        self.switch_two = False

    def itiate_movement_one(self): #player1の初期動作の関数。0,1,2,3のいずれかを返し、０が「Aを打ちやすいマルシェ」、１が「Pを打ちやすいマルシェ」、２が「Bを打ちやすいマルシェ」、３が「Rを打ちやすいマルシェ」に対応している。
        self.itiate_ugoki_one = random.choice(range(4), p=self.choice_probability_one)
        return self.itiate_ugoki_one
    
    def itiate_movement_two(self): #player2の初期動作の関数
        self.itiate_ugoki_two = random.choice(range(4), p=self.choice_probability_two)
        return self.itiate_ugoki_two

    
    def player_one(self): #[player２がだす最終動作のplayer1の予想,player1の最終動作]を返す
        self.first_movement_one = self.itiate_movement_one()
        self.first_movement_two = self.itiate_movement_two()
        
        if  (self.player_two_point-self.player_one_point) >= 3: 
            self.switch_one = True
        if self.switch_one == True:#方針転換する
            self.anticipation_one = random.choice(self.koho, p=self.disgise_two[self.first_movement_two])#"a"or"p"or"b"or"r"を返す
        else:
            if len(self.player_two_record[self.first_movement_one]) == 4:
                self.player_two_record[self.first_movement_one].pop(0)
            if (1 <= (len(self.player_two_record[self.first_movement_one]))<= 3):
                self.count_number = collections.Counter(self.player_two_record[self.first_movement_one])
                self.anticipation_one = self.gyaku_henkan[self.count_number.most_common(1)[0][0]]
            else:
                self.anticipation_one = random.choice(self.koho, p=self.disgise_two[self.first_movement_two])#"a"or"p"or"b"or"r"を返す
        #ここまでがplayer2がだす最終動作のplayer1の予想のアルゴリズム
        #「切り替え戦法（打ちやすいもの→記憶する）」を採っている
        #ここから最終動作のアルゴリズム
        self.final_reflection_probability_one = 0.5*array(self.reflection_probability_one[self.henkan[self.anticipation_one]])
        self.final_reflection_probability_one[self.first_movement_one]+= 0.5
        self.reflection_one = random.choice(self.koho, p=self.final_reflection_probability_one)
        #ここまでが最終動作のアルゴリズム
        self.result_one = [self.anticipation_one]
        self.result_one.append(self.reflection_one)
        self.aaaa= list(self.disgise_two[self.first_movement_two])#ここからの行で相手がある初動動作を出したら、その初動動作を見破る確率を上げる。
        self.aaaa.pop(self.first_movement_two)
        self.bbbb=array(self.aaaa) 
        self.disgise_two[self.first_movement_two]=insert(self.bbbb*0.8, self.first_movement_two, self.disgise_two[self.first_movement_two][self.first_movement_two]+0.2*sum(self.bbbb))
       
        return self.result_one #[player２がだす最終動作のplayer1の予想,player1の最終動作]を返す
    
    def player_two(self): #[player1がだす最終動作のplayer2の予想,player2の最終動作]を返す
        if len(self.player_one_record[self.first_movement_two]) == 4:
            self.player_one_record[self.first_movement_two].pop(0)
        if  (self.player_one_point-self.player_two_point) >= 3: 
            self.switch_two = True
        if self.switch_two == True:#方針転換する
            if (1 <= (len(self.player_one_record[self.first_movement_two]))<= 3):
                self.count_number_two = collections.Counter(self.player_one_record[self.first_movement_two])
                self.anticipation_two = self.gyaku_henkan[self.count_number_two.most_common(1)[0][0]]
            else:
                self.anticipation_two = random.choice(self.koho, p=self.disgise_two[self.first_movement_two])#"a"or"p"or"b"or"r"を返す
        else:
            self.anticipation_two =random.choice(self.koho, p=self.disgise_one[self.first_movement_one])
        #ここまでがplayer1がだす最終動作のplayer2の予想のアルゴリズム
        #「切り替え戦法（記憶する→打ちやすいもの）」を採っている
        #ここから最終動作のアルゴリズム
        self.final_reflection_probability_two = 0.5*array(self.reflection_probability_two[self.henkan[self.anticipation_two]])
        self.final_reflection_probability_two[self.first_movement_two]+= 0.5
        self.reflection_two = random.choice(self.koho, p=self.final_reflection_probability_two)
        #ここまでが最終動作のアルゴリズム
        self.result_two = [self.anticipation_two]
        self.result_two.append(self.reflection_two)
        self.cccc= list(self.disgise_one[self.first_movement_one])#ここからの行で相手がある初動動作を出したら、その初動動作を見破る確率を上げる。
        self.cccc.pop(self.first_movement_one)
        self.dddd=array(self.cccc) 
        self.disgise_one[self.first_movement_one]=insert(self.dddd*0.8, self.first_movement_one, self.disgise_one[self.first_movement_one][self.first_movement_one]+0.2*sum(self.dddd))
        self.player_one_record[self.first_movement_two].append(self.henkan[self.reflection_one])#これがplayer2の、初動動作に対する相手の反応の記憶                               
        self.player_two_record[self.first_movement_one].append(self.henkan[self.reflection_two])#これがplayer1の、初動動作に対する相手の反応の記憶                               
        
        return self.result_two  #[player1がだす最終動作のplayer2の予想,player2の最終動作]を返す



    def ritoku(self):   
        
        self.player_ichi = self.player_one()
        self.player_ni = self.player_two()
        for i in range(17):
            self.gyo_ichiran = self.ritokuhyou.cell(i,0).value
            self.gyo_ichiran = self.gyo_ichiran.split(",") 
            if self.gyo_ichiran == self.player_ichi:
                self.gyo = i
                for h in range(17):
                    self.retsu_ichiran = self.ritokuhyou.cell(0,h).value
                    self.retsu_ichiran = self.retsu_ichiran.split(",") 
                    if self.retsu_ichiran == self.player_ni:
                        self.retsu = h
        self.gyoretsu = self.ritokuhyou.cell(self.gyo,self.retsu).value
        self.shouhai_probability=self.gyoretsu.split(",")
        for i in range(3):
            self.shouhai_probability[i]= float(self.shouhai_probability[i])
        self.x=[[0,0],[1,0],[0,1]]
        self.y=[0,1,2]
        self.z=self.x[random.choice(self.y,p=self.shouhai_probability)]
        self.point.append(self.z)
        return self.point
    
    def match(self):
        while True:
            if self.player_one_point == 15:
                self.kaisu_player_one_victory += 1
                self.point = []#リセットが必要
                self.t = 1
                self.player_one_point = 0
                self.player_two_point = 0
                self.disgise_one = array([[0.5,0.5/3,0.5/3,0.5/3],[0.5/3,0.5,0.5/3,0.5/3],[0.5/3,0.5/3,0.5,0.5/3],[0.5/3,0.5/3,0.5/3,0.5]])
                self.disgise_two = array([[0.5,0.5/3,0.5/3,0.5/3],[0.5/3,0.5,0.5/3,0.5/3],[0.5/3,0.5/3,0.5,0.5/3],[0.5/3,0.5/3,0.5/3,0.5]])
                self.player_one_record= [[],[],[],[]]  
                self.player_two_record= [[],[],[],[]]  #APBRの順  
                self.switch_one = False
                self.switch_two = False
                break
            elif self.player_two_point == 15:   
                self.point = []
                self.t = 1
                self.player_one_point = 0
                self.player_two_point = 0
                self.disgise_one = array([[0.5,0.5/3,0.5/3,0.5/3],[0.5/3,0.5,0.5/3,0.5/3],[0.5/3,0.5/3,0.5,0.5/3],[0.5/3,0.5/3,0.5/3,0.5]])
                self.disgise_two = array([[0.5,0.5/3,0.5/3,0.5/3],[0.5/3,0.5,0.5/3,0.5/3],[0.5/3,0.5/3,0.5,0.5/3],[0.5/3,0.5/3,0.5/3,0.5]])
                self.player_one_record= [[],[],[],[]] 
                self.player_two_record= [[],[],[],[]]  #APBRの順  
                self.switch_one = False
                self.switch_two = False
                break
            else:
                self.game = self.ritoku()
                if  self.game[self.t-1] == [1,0]:
                    self.player_one_point += 1
                if  self.game[self.t-1] == [0,1]:
                    self.player_two_point += 1
                self.t += 1
        
        return self.point
        
    def game_results(self): #player1の勝利回数。
        for i in range(10):#試合数
            self.kurikaesu = self.match()
            #self.match()
        return self.kaisu_player_one_victory
    def game_results_heikin(self): #player1の勝利回数の平均。
        self.hanpuku = 0
        for i in range(100):#繰り返す回数#100なら安定した。
            self.tameshi = self.game_results()
            #print(self.tameshi)
            self.hanpuku += self.tameshi
            self.kaisu_player_one_victory = 0
            self.heikin = self.hanpuku/100
        return  self.heikin
kurikaeshi= Fencing(0.25,0.25,0.25,0.25,0.25,0.25)
print(kurikaeshi.game_results_heikin())

4.17
